In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

tf.__version__

'2.1.0'

### 1. tensorflow hello world

最简单的神经网络、一层、一个单元、一个输入

In [ ]:
xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-2.0, 1.0, 4.0, 7.0, 10.0, 13.0], dtype=float)

model = keras.Sequential()

# 添加全连接层
model.add(keras.layers.Dense(units=1, input_shape=[1]))

# 指定损失函数 和 优化器
model.compile(optimizer='sgd', loss='mean_squared_error')

# 训练
model.fit(xs, ys, epochs=100)

# 预测
model.predict([10.0])

### 2. 手写数字识别

In [4]:
mnist = tf.keras.datasets.mnist

In [30]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [27]:
print(f'training data len: {len(x_train)}')
print(f'testing data len: {len(x_test)}')

training data len: 60000
testing data len: 10000


下面是做了数据归一化处理

数据的标准化（normalization）是将数据按比例缩放，使之落入一个小的特定区间。
数据标准化最典型的就是数据的归一化处理，即将数据统一映射到[0,1]区间上。从经验上说，归一化是让不同维度之间的特征在数值上有一定比较性，可以大大提高分类器的准确性。

归一化好处：
1. 提升模型的收敛速度
2. 提升模型的精度

如果多了这一步，准确率达到97.6%， 少了这步准确率不到95%

In [33]:
# x_train, x_test = x_train / 255.0, x_test / 255.0  # 数据归一化

In [25]:
# 取一张图来查看下
# 每一张图都是28 * 28像素的二值图像
from PIL import Image

print(y_train[0])
Image.fromarray(x_train[0]).convert('L')

5


In [31]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 60us/sample - loss: 0.0313 - accuracy: 0.9894
Epoch 2/5
60000/60000 [==============================] - 3s 52us/sample - loss: 0.0285 - accuracy: 0.9905
Epoch 3/5
60000/60000 [==============================] - 3s 52us/sample - loss: 0.0287 - accuracy: 0.9901
Epoch 4/5
60000/60000 [==============================] - 3s 52us/sample - loss: 0.0281 - accuracy: 0.9907
Epoch 5/5
60000/60000 [==============================] - 3s 51us/sample - loss: 0.0252 - accuracy: 0.9916
10000/10000 - 0s - loss: 0.0851 - accuracy: 0.9806


[0.08514741737058502, 0.9806]

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
